# Daten der Verkehrszählung zum motorisierten Individualverkehr (MIV)

Dieses Notebook zeigt einige Beispiele, wie mit den Daten aus dem Datenatz [Daten der Verkehrszählung zum motorisierten Individualverkehr (Stundenwerte), seit 2012](https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031) gearbeitet werden kann.

Der Datensatz wird **täglich** aktualisiert und beinhaltet jeweils die Daten bis und mit zum vorletzten Tag (z.B. der Datensatz vom 16.3.2020 beinhaltet die Daten bis und mit 14.3.2020.

## Inhaltsverzeichnis

1. [Datei mit Pandas einlesen](#Datei-mit-Pandas-einlesen)
2. [Übersicht der Zählstellen und deren Position (Karte)](#%C3%9Cbersicht-der-Z%C3%A4hlstellen-und-deren-Position-(Karte))
3. Einfache Visualisierung einer Zählstelle über einen Zeitraum
4. Vergleich der Werte einer Zählstelle zwischen mehreren Wochen

In [1]:
import requests
import pandas as pd

## Datei mit Pandas einlesen

2020er Datei vom OGD-Katalog herunterladen:

In [2]:
result = requests.get(
    'https://data.stadt-zuerich.ch/api/3/action/package_show?id=sid_dav_verkehrszaehlung_miv_od2031'
)
dataset = result.json()['result']
traffic_url = None
for r in dataset['resources']:
    if r['name'] == 'sid_dav_verkehrszaehlung_miv_OD2031_2020.csv':
        traffic_url = r['url']
        break

assert traffic_url, "Datei sid_dav_verkehrszaehlung_miv_OD2031_2020.csv nicht gefunden."
df = pd.read_csv(traffic_url, encoding='utf-8')
df

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,...,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T00:00:00,2020-03-15,247.0,Gemessen
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T01:00:00,2020-03-15,364.0,Gemessen
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T02:00:00,2020-03-15,159.0,Gemessen
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T03:00:00,2020-03-15,108.0,Gemessen
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T04:00:00,2020-03-15,87.0,Gemessen
5,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T05:00:00,2020-03-15,55.0,Gemessen
6,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T06:00:00,2020-03-15,41.0,Gemessen
7,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T07:00:00,2020-03-15,44.0,Gemessen
8,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T08:00:00,2020-03-15,40.0,Gemessen
9,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T09:00:00,2020-03-15,43.0,Gemessen


In [3]:
df.columns

Index(['MSID', 'MSName', 'ZSID', 'ZSName', 'Achse', 'HNr', 'Hoehe', 'EKoord',
       'NKoord', 'Richtung', 'Knummer', 'Kname', 'AnzDetektoren', 'D1ID',
       'D2ID', 'D3ID', 'D4ID', 'MessungDatZeit', 'LieferDat', 'AnzFahrzeuge',
       'AnzFahrzeugeStatus'],
      dtype='object')

## Übersicht der Zählstellen und deren Position (Karte)

In [5]:
zaehlstellen = df[['MSID', 'MSName', 'ZSID', 'ZSName', 'Achse', 'HNr', 'Hoehe', 'EKoord',
       'NKoord', 'Richtung']]
zaehlstellen = zaehlstellen.drop_duplicates().reset_index(drop=True)
zaehlstellen

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts
1,Z001M002,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683013.18,1243937.64,einwärts
2,Z002M001,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682480.51,1243162.50,auswärts
3,Z002M002,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682484.91,1243162.41,einwärts
4,Z003M001,Unbekannt,Z003,Sood-/Leimbachstrasse,Soodstrasse,Unbekannt,Leimbachstrasse,2681573.79,1242969.91,auswärts
5,Z003M002,Unbekannt,Z003,Sood-/Leimbachstrasse,Leimbachstrasse,64,Unbekannt,2681607.86,1243010.95,einwärts
6,Z004M001,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,90m ab Knoten,2679762.52,1246863.95,auswärts
7,Z004M002,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,vor Knoten,2679771.84,1246852.29,einwärts
8,Z005M001,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678874.03,1247598.55,auswärts
9,Z005M002,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678871.97,1247592.79,einwärts
